# Notebook for analysing the univariate stats of all tables

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-dark')
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
imp_tables = [
    'encounters',
    'patients',
    'careplans',
    'conditions',
    'medications',
    'observations',
    'organizations',
    'procedures',
    'providers'
]

In [3]:
dict = {}
for tables in imp_tables:
    query = f'''
        select count(*) as row_count from {tables};
    '''
    df = sql(query)
    dict[tables] = df['row_count'][0]

pd.DataFrame.from_dict(dict,orient='index',columns=['row_count']).reset_index().rename(columns={'index':'table_name'})

,table_name,row_count
0,encounters,53346
1,patients,1171
2,careplans,3483
3,conditions,8376
4,medications,42989
5,observations,299697
6,organizations,1119
7,procedures,34981
8,providers,5855


`SO IT DOES NOT MEAN THAT EVERY ENCOUNTER CAN HAVE PROCEDURES, MEDICATIONS, CAREPLANS, OR DETECTED CONDITION`

## Encounters

## Patients

## Careplans

## Conditions

## Medications

## Observations

## Organizations

## Procedures

## Providers